In [237]:
import glob
from utils import in_blocklist

blocklist = []
blocklist.append('AWAYLEN=307 MAXTARGETS=20 WALLCHOPS WATCH=128 WATCHOPTS=A SILENCE=15 MODES=12 CHANTYPES=# PREFIX')
blocklist.append('- * -')
blocklist.append('XxXChatters.Com')
blocklist.append('XxXChatters')
blocklist.append('This server was created')
blocklist.append('operator(s) online')
blocklist.append('is now your displayed host')
blocklist.append('Caps set:')
blocklist.append('MAXCHANNELS')
blocklist.append('http')
blocklist.append('type it in the main room or a pm to the')
blocklist.append('The topic is')
blocklist.append('Topic set by')


all_posts = []
all_posts_410 = []
all_posts_501 = []
authors = {}
authors_410 = {}
authors_501 = {}

for file in glob.glob("xxxchatters_logs/#chat*.log"):
    lines = open(file, 'r', encoding='utf-8').readlines()
    
    for line in lines:
        if not in_blocklist(blocklist, line.strip()): # blocked?
            if line[0] is '[':
                s = line
                if s.find("<") is not -1:
                    sentence = s[s.find(">")+2:]
                    
                    author = s[s.find("<")+1:s.find(">")]
                    
                    all_posts.append((author, sentence.strip().lower()))
                    
                    if author not in authors: # save things
                        authors[author] = set() # init list for author
                    authors[author].add(sentence.strip().lower()) 
            

for line in open("1006/410.txt", 'r').readlines():
    if ':' in line and not in_blocklist(blocklist, line.strip()): # blocked?:
        sentence = line[line.find(":"):]
        all_posts.append((author, sentence.strip().lower()))
        all_posts_410.append((author, sentence.strip().lower()))
        
        author = line[:line.find(":")]
        if author not in authors: # save things
            authors[author] = set() # init list for author
        if author not in authors_410:
            authors_410[author] = set()
        authors[author].add(sentence.strip().lower())
        authors_410[author].add(sentence.strip().lower()) 


for s in open("1006/501.txt", 'r').readlines():
    if ':' in s and 'has left the channel' not in s and 'has quit' not in s and 'joined the channel' not in s:
        if not in_blocklist(blocklist, s.strip()): # blocked?
            sentence = s[s.find(">")+2:]
            all_posts.append((author, sentence.strip().lower()))
            all_posts_501.append((author, sentence.strip().lower()))

            author = s[s.find("<")+1:s.find(">")]
            if author not in authors: # save things
                authors[author] = set() # init list for author
            if author not in authors_501:
                authors_501[author] = set()
            authors[author].add(sentence.strip().lower())
            authors_501[author].add(sentence.strip().lower()) 
        
print(len(all_posts))
print(len(authors))
print(len(authors_410))
print(len(authors_501))
print(len(all_posts_410))
print(len(all_posts_501))

xxxchatters_logs\#chat.20180214.log
xxxchatters_logs\#chat.20180215.log
xxxchatters_logs\#chat.20180216.log
xxxchatters_logs\#chat.20180217.log
xxxchatters_logs\#chat.20180219.log
xxxchatters_logs\#chat.20180220.log
xxxchatters_logs\#chat.20180221.log
xxxchatters_logs\#chat.20180222.log
xxxchatters_logs\#chat.20180223.log
xxxchatters_logs\#chat.20180224.log
xxxchatters_logs\#chat.20180225.log
xxxchatters_logs\#chat.20180226.log
xxxchatters_logs\#chat.20180227.log
xxxchatters_logs\#chat.20180228.log
xxxchatters_logs\#chat.20180301.log
xxxchatters_logs\#chat.20180302.log
xxxchatters_logs\#chat.20180303.log
xxxchatters_logs\#chat.20180304.log
xxxchatters_logs\#chat.20180305.log
xxxchatters_logs\#chat.20180306.log
86368
1909
40
73
304
238


In [ ]:
for a in authors['carrol']:
    print(a)

In [238]:
import operator
import string
word_counts = dict()
translator = str.maketrans('', '', string.punctuation)

def preprocess_word(word):
    word = word.replace('\x02', '')
    word = word.replace('\x03', '')
    word = word.replace('.', '')

    #word = word.translate(translator)
    word = ''.join(e for e in word if e.isalnum())
    return word


for post in all_posts:
    word_list = post[1].split()
    for word in word_list:
        word = preprocess_word(word)
        #word = word.replace('\x03', '')
        #word = word.replace('.', '')

        #word = word.translate(translator)
        #word = ''.join(e for e in word if e.isalnum())
        if len(word) < 1:
            continue
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

sorted_word_counts = sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)

def get_sorted_word_counts():
    return sorted_word_counts

v = get_sorted_word_counts()
print(len(v))

30389


In [239]:
def get_topn_percent(perc):
    counts = get_sorted_word_counts()
    l = len(counts)
    max_index = int(perc*l)
    top_n = []
    print(counts[:max_index])
    for t in counts[:max_index]:
        top_n.append(t[0])
    return top_n

def get_all_by_max_appearance(app):
    counts = get_sorted_word_counts()
    processed_counts = []
    for c in counts:
        if c[1] >= app:
            processed_counts.append(c)
        
    return processed_counts
    
#topn = get_topn_percent(0.001)
maxapp = get_all_by_max_appearance(2)
file = open("1106/chat_room_word_appearance.txt", 'w')
for e in maxapp:
    try:
        file.write(e[0] + ", " + str(e[1]) + '\n')
    except:
        pass
file.close()
#print(maxapp)
#print(topn)
#print(len(topn))

In [213]:
def remove_non_frequent(allowed_words, post, replace=True):
    good_words = []
    for word in post.lower().split():
        pword = preprocess_word(word)
        if pword in allowed_words:
            good_words.append(pword)
        else:
            if replace:
                replacement = ""
                for char in word:
                    replacement += "-"
                good_words.append(replacement)
    
    return " ".join(good_words)


sentence_to_test = "Jus to keep being evil<<<< wops Athena on the backside with a new nerf bat"
print(remove_non_frequent(get_topn_percent(0.5), sentence_to_test))

jus to keep being evil wops athena on the backside with a new nerf bat


In [214]:
sentence_to_test = "bops saffron on the backside with one of my new nerf bats"

v = 0.0
while v <= 1.0:
    v += 0.04
    print(remove_non_frequent(get_topn_percent(v), sentence_to_test))
    

---- saffron on the -------- with one of my new ---- ----
---- saffron on the -------- with one of my new nerf ----
bops saffron on the -------- with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron o

In [215]:
import numpy as np

def get_by_probability(authors, count, topn = 30):
    author_probability = {}
    
    for key, value in authors.items():
        author_probability[key] = len(value)

    author_probability_counts = sorted(author_probability.items(), key=operator.itemgetter(1), reverse=True)
    
    #print(author_probability_counts[:topn])

    sum_sentences = 0
    for v in author_probability_counts[:topn]:
        sum_sentences += v[1]

    author_probability_final = {}
    author_list = []
    prob_list = []

    for v in author_probability_counts[:topn]:
        author_probability_final[v[0]] = v[1]/float(sum_sentences)
        author_list.append(v[0])
        prob_list.append(v[1]/float(sum_sentences))

    final_autor_list = np.random.choice(
      author_list, 
      count,
      p=prob_list
    )

    return final_autor_list

print(get_by_probability(authors, 20))
print(get_by_probability(authors_410, 20))
print(get_by_probability(authors_501, 20))

['DJ`Mercury' 'Anastatia' '@pwincess_aria' 'JustMe' 'Auctioneer' 'JustMe'
 '@Oscar' 'Malex' 'Gramps' 'DJ`Mercury' '+DJ`Mercury' 'Wolf-in-the-alley'
 '&annora' '@Oscar' 'simian' 'Valkyrie' '&Karman' '&annora'
 'slave_of_denial_m' 'Athena']
['carrol' '^fran' 'Anastatia' 'Woman' '+DJ`South' '+DJ`Mercury' 'Valkyrie'
 'carrol' 'carrol' 'Threeleggedcat' 'JFetish' 'Louise' 'carrol' 'Louise'
 'PlayfulBBC' 'Woman' 'Louise' 'Anastatia' 'PlayfulBBC' 'WhoGiveSaDamn']
['@erotic_kitty' '@erotic_kitty{S}' '@erotic_kitty' '+DJ`Mercury'
 'keynotespkr' 'FunkyBoogieKing' '@erotic_kitty' 'FriendlyMonster'
 '@erotic_kitty' 'MeanMark1' 'sweet_teresa' 'FriendlyMonster'
 '@erotic_kitty' '@erotic_kitty' '@erotic_kitty' '@DJ`liltech'
 '@erotic_kitty{S}' '@erotic_kitty' 'Sanger' 'keynotespkr']


In [217]:
import random

def generate_lines_linear(count):
    for i in range(1, count):
        sentence = random.choice(all_posts)
        perc = i/float(count)

        topn = get_topn_percent(perc)
        stripped_post = remove_non_frequent(topn, sentence.strip().lower())
        print(str(perc) + ": " + stripped_post)
        
def generate_lines_from(fromp, step, count):
    for i in range(count):
        sentence = random.choice(all_posts)
        perc = fromp + i * step

        topn = get_topn_percent(perc)
        #print(len(topn))
        stripped_post = remove_non_frequent(topn, sentence[1].strip().lower())
        print("{:.6f}".format(perc) + ": " + stripped_post)
        
#generate_lines_linear(20)

generate_lines_from(0.00, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.10, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.20, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.50, 0.0001, 10)
print("####################################################################################")
generate_lines_from(0.80, 0.0001, 10)

0.000000: --- ------ ---- ---- ---- --- ----- ---
0.000100: ---- --- --- --------
0.000200: --- ----- -----
0.000300: a ----- -- ----- -----
0.000400: ----- -----
0.000500: ------------------------------ ---------
0.000600: and -- to the --------
0.000700: ----
0.000800: --- -------- ----- ------------
0.000900: 14 radio meltdown djmercury is playing 6 rm -- promo - ------
####################################################################################
0.100000: carrot me oscar
0.100100: hi m45canada
0.100200: 14 radio meltdown djmercury is playing 6 lenny kravitz - love revolution
0.100300: there was -----
0.100400: why do you gotta play hard to get with me octa
0.100500: i said yes
0.100600: ---------------------- that nick ----- ------- down my ------
0.100700: lol
0.100800: hi obiwan
0.100900: morning
####################################################################################
0.200000: 70hi glenn welcome to pleasurelibrarydungeon where we openly invite scening
0.200100

In [223]:
# postprocess act1&2

for sentence in all_posts_501:
    print(sentence[0] + " -> " + remove_non_frequent(get_topn_percent(0.01), sentence[1]))

Valkyrie -> hello ------------ sir
onetoquiet -> shes ----- ----- ---------------
FunkyBoogieKing -> heya ------------
onetoquiet -> hiyas ------------ ------------- and --------
@erotic_kitty -> heya ------------ ty
Violatef -> hi ------------
jakkiline-cd -> ---------
@erotic_kitty -> oh ------------ why
Violatef -> welcome --------
@erotic_kitty -> wb ------
@erotic_kitty -> radio meltdown djliltech is playing --- ---- ------- - --- for my life
+DJ`Mercury -> --------- ------------ --------- who the -------- and who the --------
[23:04:08] Random_Chaos wondering who the violator and who the violatee -> have a great night
WaywardGentleman -> --------- funkyboogieking --------------- ------------
[23:04:41] FunkyBoogieKing pounce-whomples erotic_kitty -> --------------------------
@erotic_kitty{S} -> while the ----------- ----- is away the ----------- ------ will play
FunkyBoogieKing -> -----
@erotic_kitty{S} -> wb -------
@erotic_kitty{S} -> ty -----
JoyceGG -> --------- ----- ----- 